In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
file_path = '/content/drive/Shareddrives/dm_final/dm_final/data/gridcentroid_dataframe_07x07_weekday.csv.xls'
data = pd.read_csv(file_path)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Selecting relevant features and the target variable
features = data[['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart',
                 'IsWeekend', 'GridID', 'X_centroid', 'Y_centroid']]
target = data['NumberOfIncidents']

# Simplified parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 150],
    'learning_rate': [0.1, 0.15],
    'max_depth': [3, 4]
}

# Setting up GridSearchCV
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                           param_grid=param_grid,
                           scoring='r2',
                           cv=3,
                           verbose=1,
                           n_jobs=-1)

# Fitting GridSearchCV to the data
grid_search.fit(features,target)

# Best parameters and score from the grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Training the model with the best parameters
best_gb_model = GradientBoostingRegressor(**best_params, random_state=42)
best_gb_model.fit(features, target)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GradientBoostingRegressor(max_depth=4, random_state=42)

In [ ]:
print("Best Parameters:", best_params)
print("Best Grid Search Score:", best_score)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Best Grid Search Score: -0.4588588837624537


In [ ]:
import pandas as pd
file_path = '/content/drive/Shareddrives/dm_final/dm_final/data/test_gridcentroid_test_07x07_weekday.csv.xls'
test_data = pd.read_csv(file_path)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Re-train the Gradient Boosting Regressor with the best parameters
# Preparing the test data for prediction
X_test = test_data[['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart',
                    'IsWeekend', 'GridID', 'X_centroid', 'Y_centroid']]
y_test = test_data['NumberOfIncidents']

# Making predictions on the test data
y_pred_test = best_gb_model.predict(X_test)

# Calculating performance metrics
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

mse_test, rmse_test, mae_test, r2_test

(20.73233799141072, 4.553277719556618, 2.829004603655231, 0.7097011685994772)

In [ ]:
y_pred_test

array([ 1.75010357,  1.66669569,  1.6072216 ,  1.6669593 ,  1.81176513,
        1.81234012,  1.46626752,  1.91389356,  1.91389356,  1.91298449,
        1.91298449,  1.85646942,  1.83291391,  1.85190573,  1.85190573,
        1.85190573,  2.35861486,  2.27520697,  2.21573289,  2.21573289,
        2.27547058,  2.42027642,  2.0747788 ,  2.54401586,  2.54310678,
        2.54310678,  2.48659171,  2.44142519,  2.44142519,  2.44142519,
        2.46041701,  2.46041701,  2.46041701,  6.5124177 ,  6.42900981,
        6.36953573,  6.36953573,  6.45971569,  6.60452152,  6.77623334,
        6.57465425,  6.22858164,  6.73262739,  6.73262739,  6.76216057,
        6.76216057,  6.76216057,  6.67520324,  6.59522804,  6.59522804,
        6.59522804,  6.64466211,  6.64466211,  6.64466211,  5.60892946,
        5.52552157,  5.46604749,  5.46604749,  5.55622745,  5.70103328,
        5.8727451 ,  5.67116601,  5.3250934 ,  5.78670396,  5.78670396,
        5.81623714,  5.81623714,  5.81623714,  5.72927981,  5.64

In [ ]:
import pandas as pd

y_pred_series = pd.Series(y_pred_test, name='PredictedNumberOfIncidents')

# Merge the predictions with the X_test dataframe
merged_df = pd.concat([X_test, y_pred_series], axis=1)

# Now merged_df contains both the features and the predicted number of incidents
print(merged_df.head())

   OccurrenceYear  OccurrenceMonth  OccurrenceDatePart  IsWeekend  GridID  \
0            2023               10                  23          0       1   
1            2023               10                  24          0       1   
2            2023               10                  25          0       1   
3            2023               10                  27          1       1   
4            2023               10                  28          1       1   

   X_centroid  Y_centroid  PredictedNumberOfIncidents  
0 -111.971255   33.330417                    1.750104  
1 -111.971255   33.330417                    1.666696  
2 -111.971255   33.330417                    1.607222  
3 -111.971255   33.330417                    1.666959  
4 -111.971255   33.330417                    1.811765  


In [ ]:
updated_dataframe_path = '/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted.csv'
merged_df.to_csv(updated_dataframe_path, index=False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# Extract unique dates and grid IDs
unique_dates_final_predicted = final_predicted_df['OccurrenceDatePart'].unique()
unique_grid_ids = grid_mapping_df['GridID'].unique()

# Initialize an empty DataFrame to store the updated data
updated_final_predicted_df = pd.DataFrame()

for date in unique_dates_final_predicted:
    for grid_id in unique_grid_ids:
        existing_record = final_predicted_df[(final_predicted_df['OccurrenceDatePart'] == date) & (final_predicted_df['GridID'] == grid_id)]

        if existing_record.empty:
            grid_info = grid_mapping_df[grid_mapping_df['GridID'] == grid_id]
            new_record = {
                'OccurrenceYear': existing_record.iloc[0]['OccurrenceYear'] if not existing_record.empty else 2023,
                'OccurrenceMonth': existing_record.iloc[0]['OccurrenceMonth'] if not existing_record.empty else 10,
                'OccurrenceDatePart': date,
                'IsWeekend': existing_record.iloc[0]['IsWeekend'] if not existing_record.empty else 0,
                'GridID': grid_id,
                'X_centroid': grid_info.iloc[0]['X_centroid'] if 'X_centroid' in grid_info.columns else None,
                'Y_centroid': grid_info.iloc[0]['Y_centroid'] if 'Y_centroid' in grid_info.columns else None,
                'PredictedNumberOfIncidents': 0,
                'geometry': grid_info.iloc[0]['geometry']
            }
            updated_final_predicted_df = updated_final_predicted_df.append(new_record, ignore_index=True)
        else:
            updated_final_predicted_df = updated_final_predicted_df.append(existing_record)

# Resetting index of the updated DataFrame
updated_final_predicted_df.reset_index(drop=True, inplace=True)
updated_final_predicted_df

,OccurrenceYear,OccurrenceMonth,OccurrenceDatePart,IsWeekend,GridID,X_centroid,Y_centroid,PredictedNumberOfIncidents,geometry
0,2023,10,23,0,1,-111.971255,33.330417,1.750104,NaN
1,2023,10,23,0,2,-111.971255,33.35117,2.358615,NaN
2,2023,10,23,0,3,-111.971255,33.371923,6.512418,NaN
3,2023,10,23,0,4,-111.971255,33.392677,5.608929,NaN
4,2023,10,23,0,5,-111.971255,33.41343,6.592475,NaN
...,...,...,...,...,...,...,...,...,...
963,2023,10,13,0,40,None,None,0.000000,POLYGON ((-111.89183343328567 33.4238069181428...
964,2023,10,13,0,41,None,None,0.000000,POLYGON ((-111.87739323399995 33.3615467708571...
965,2023,10,13,0,42,None,None,0.000000,POLYGON ((-111.87739323399995 33.3823001532857...
966,2023,10,13,0,43,None,None,0.000000,POLYGON ((-111.87739323399995 33.4030535357143...


In [ ]:
updated_final_predicted_df.drop(columns=['geometry'])
updated_final_predicted_df.head(100)

,OccurrenceYear,OccurrenceMonth,OccurrenceDatePart,IsWeekend,GridID,X_centroid,Y_centroid,PredictedNumberOfIncidents,geometry
0,2023,10,23,0,1,-111.971255,33.330417,1.750104,POLYGON ((-111.96403442971423 33.3200400060000...
1,2023,10,23,0,2,-111.971255,33.35117,2.358615,POLYGON ((-111.96403442971423 33.3407933884286...
2,2023,10,23,0,3,-111.971255,33.371923,6.512418,POLYGON ((-111.96403442971423 33.3615467708571...
3,2023,10,23,0,4,-111.971255,33.392677,5.608929,POLYGON ((-111.96403442971423 33.3823001532857...
4,2023,10,23,0,5,-111.971255,33.41343,6.592475,POLYGON ((-111.96403442971423 33.4030535357143...
...,...,...,...,...,...,...,...,...,...
95,2023,10,25,0,8,-111.956814,33.35117,7.896778,POLYGON ((-111.94959423042852 33.3407933884286...
96,2023,10,25,0,9,-111.956814,33.371923,11.160245,POLYGON ((-111.94959423042852 33.3615467708571...
97,2023,10,25,0,10,-111.956814,33.392677,8.687680,POLYGON ((-111.94959423042852 33.3823001532857...
98,2023,10,25,0,11,-111.956814,33.41343,14.719144,POLYGON ((-111.94959423042852 33.4030535357143...


In [ ]:
import pandas as pd
from itertools import product

final_predicted_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted.csv')
grid_mapping_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/grid_mapping_7x7.csv')

# Extract unique values for year, month, and date part
unique_years = final_predicted_df['OccurrenceYear'].unique()
unique_months = final_predicted_df['OccurrenceMonth'].unique()
unique_date_parts = final_predicted_df['OccurrenceDatePart'].unique()

# Extract grid IDs from the grid mapping file
unique_grid_ids = grid_mapping_df['GridID'].unique()

# Generate all possible combinations
all_combinations = list(product(unique_years, unique_months, unique_date_parts, unique_grid_ids))

# Convert these combinations into a DataFrame
all_combinations_df = pd.DataFrame(all_combinations, columns=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'])

# Identify missing combinations in final_predicted.csv
missing_combinations_df = all_combinations_df[~all_combinations_df.isin(final_predicted_df).all(axis=1)]

# Create a new DataFrame with missing combinations and incidents set to 0
missing_combinations_df['PredictedNumberOfIncidents'] = 0

# Merge the original data with the missing combinations
complete_data_df = pd.concat([final_predicted_df, missing_combinations_df], ignore_index=True)

# Sort the data for better organization
complete_data_df.sort_values(by=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'], inplace=True)

# Reset the index
complete_data_df.reset_index(drop=True, inplace=True)

# Now complete_data_df contains the full dataset with missing entries filled
complete_data_df

,OccurrenceYear,OccurrenceMonth,OccurrenceDatePart,IsWeekend,GridID,X_centroid,Y_centroid,PredictedNumberOfIncidents
0,2023,10,1,NaN,1,NaN,NaN,0.0
1,2023,10,1,NaN,2,NaN,NaN,0.0
2,2023,10,1,NaN,3,NaN,NaN,0.0
3,2023,10,1,NaN,4,NaN,NaN,0.0
4,2023,10,1,NaN,5,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2753,2023,11,31,NaN,40,NaN,NaN,0.0
2754,2023,11,31,NaN,41,NaN,NaN,0.0
2755,2023,11,31,NaN,42,NaN,NaN,0.0
2756,2023,11,31,NaN,43,NaN,NaN,0.0


In [ ]:
import pandas as pd
from itertools import product

final_predicted_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted.csv')
grid_mapping_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/grid_mapping_7x7.csv')

# Extract unique values for year, month, and date part
unique_years = final_predicted_df['OccurrenceYear'].unique()
unique_months = final_predicted_df['OccurrenceMonth'].unique()
unique_date_parts = final_predicted_df['OccurrenceDatePart'].unique()

# Extract grid IDs from the grid mapping file
unique_grid_ids = grid_mapping_df['GridID'].unique()

# Generate all possible combinations
all_combinations = list(product(unique_years, unique_months, unique_date_parts, unique_grid_ids))

# Convert these combinations into a DataFrame
all_combinations_df = pd.DataFrame(all_combinations, columns=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'])

# Merge the original data with all possible combinations using a left join
complete_data_df = pd.merge(all_combinations_df, final_predicted_df,
                            on=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'],
                            how='left')

# Fill NaN values in PredictedNumberOfIncidents with 0
complete_data_df['PredictedNumberOfIncidents'].fillna(0, inplace=True)

# Sort the data for better organization
complete_data_df.sort_values(by=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'], inplace=True)

# Reset the index
complete_data_df.reset_index(drop=True, inplace=True)

# Now complete_data_df contains the full dataset with original values

In [ ]:
complete_data_df

,OccurrenceYear,OccurrenceMonth,OccurrenceDatePart,GridID,IsWeekend,X_centroid,Y_centroid,PredictedNumberOfIncidents
0,2023,10,1,1,NaN,NaN,NaN,0.0
1,2023,10,1,2,NaN,NaN,NaN,0.0
2,2023,10,1,3,NaN,NaN,NaN,0.0
3,2023,10,1,4,NaN,NaN,NaN,0.0
4,2023,10,1,5,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
1931,2023,11,31,40,NaN,NaN,NaN,0.0
1932,2023,11,31,41,NaN,NaN,NaN,0.0
1933,2023,11,31,42,NaN,NaN,NaN,0.0
1934,2023,11,31,43,NaN,NaN,NaN,0.0


In [ ]:
import pandas as pd
from itertools import product

final_predicted_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted.csv')
grid_mapping_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/grid_mapping_7x7.csv')

# Extract unique values for year, month, and date part from final_predicted.csv
unique_years_months_dates = final_predicted_df[['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart']].drop_duplicates()

# Extract grid IDs from the grid mapping file
unique_grid_ids = grid_mapping_df['GridID'].unique()

# Generate all possible combinations of year, month, date part, and grid ID
all_combinations = product(unique_years_months_dates.itertuples(index=False), unique_grid_ids)
all_combinations_df = pd.DataFrame([(*ymd, grid_id) for ymd, grid_id in all_combinations],
                                   columns=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'])

# Merge the original data with all possible combinations using a left join
complete_data_df = pd.merge(all_combinations_df, final_predicted_df,
                            on=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'],
                            how='left')

# Fill NaN values in PredictedNumberOfIncidents with 0
complete_data_df['PredictedNumberOfIncidents'].fillna(0, inplace=True)

# Sort the data for better organization
complete_data_df.sort_values(by=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'], inplace=True)

# Reset the index
complete_data_df.reset_index(drop=True, inplace=True)

In [ ]:
import pandas as pd
from itertools import product

final_predicted_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted.csv')
grid_mapping_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/grid_mapping_7x7.csv')

# Extract unique values for year, month, and date part from final_predicted.csv
unique_years_months_dates = final_predicted_df[['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart']].drop_duplicates()

# Extract grid IDs from the grid mapping file
unique_grid_ids = grid_mapping_df['GridID'].unique()

# Generate all possible combinations of year, month, date part, and grid ID
all_combinations = product(unique_years_months_dates.itertuples(index=False), unique_grid_ids)
all_combinations_df = pd.DataFrame([(*ymd, grid_id) for ymd, grid_id in all_combinations],
                                   columns=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'])

# Merge the original data with all possible combinations using a left join
complete_data_df = pd.merge(all_combinations_df, final_predicted_df,
                            on=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'],
                            how='left')

# Fill NaN values in PredictedNumberOfIncidents with 0
complete_data_df['PredictedNumberOfIncidents'].fillna(0, inplace=True)

# Join with grid mapping data based on GridID
complete_data_with_grid_info_df = pd.merge(complete_data_df, grid_mapping_df, on='GridID', how='left')

# Sort the data for better organization
complete_data_with_grid_info_df.sort_values(by=['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart', 'GridID'], inplace=True)

# Reset the index
complete_data_with_grid_info_df.reset_index(drop=True, inplace=True)

# Now complete_data_with_grid_info_df contains the full dataset with grid mapping information

In [ ]:
updated_dataframe_path = '/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted_grid.csv'
complete_data_with_grid_info_df.to_csv(updated_dataframe_path, index=False)

In [ ]:
import pandas as pd
import folium
from shapely.wkt import loads
import imageio
import os

final_predicted_grid_df = pd.read_csv('/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted_grid.csv')

# Convert the geometry column from WKT to Shapely objects
final_predicted_grid_df['geometry'] = final_predicted_grid_df['geometry'].apply(loads)

# Get unique dates from the dataset
unique_dates = final_predicted_grid_df[['OccurrenceYear', 'OccurrenceMonth', 'OccurrenceDatePart']].drop_duplicates()

map_files = []

for _, unique_date in unique_dates.iterrows():
    # Filter data for the specific day
    day_data = final_predicted_grid_df[
        (final_predicted_grid_df['OccurrenceYear'] == unique_date['OccurrenceYear']) &
        (final_predicted_grid_df['OccurrenceMonth'] == unique_date['OccurrenceMonth']) &
        (final_predicted_grid_df['OccurrenceDatePart'] == unique_date['OccurrenceDatePart'])
    ]

    # Initialize map
    day_map = folium.Map(location=[33.4255, -111.9400], zoom_start=12)

    # Plot each grid's polygon and incidents on the map
    for _, row in day_data.iterrows():
        folium.GeoJson(
            row['geometry'],
            tooltip=f"Grid ID: {row['GridID']}<br>Incidents: {row['PredictedNumberOfIncidents']}"
        ).add_to(day_map)

    map_file = f'tempe_map_{unique_date["OccurrenceYear"]}_{unique_date["OccurrenceMonth"]:02d}_{unique_date["OccurrenceDatePart"]:02d}.html'
    day_map.save(map_file)
    map_files.append(map_file)

for filename in map_files:
    os.remove(filename)

In [ ]:
for filename in map_files:
    os.remove(filename)

In [ ]:
import plotly.express as px
import geopandas as gpd
import pandas as pd

file_path = '/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted_grid.csv'
data = pd.read_csv(file_path)

# Converting the 'geometry' column to a GeoDataFrame
data['geometry'] = gpd.GeoSeries.from_wkt(data['geometry'])
geo_data = gpd.GeoDataFrame(data, geometry='geometry')

# Function to create an interactive map for a selected date
def plot_interactive_map(selected_year, selected_month, selected_day):
    day_data = geo_data[(geo_data['OccurrenceYear'] == selected_year) &
                        (geo_data['OccurrenceMonth'] == selected_month) &
                        (geo_data['OccurrenceDatePart'] == selected_day)]

    fig = px.choropleth_mapbox(day_data, geojson=day_data.geometry.__geo_interface__,
                               locations=day_data.index, color='PredictedNumberOfIncidents',
                               color_continuous_scale="OrRd",
                               mapbox_style="carto-positron",
                               zoom=10, center={"lat": day_data.geometry.centroid.y.mean(),
                                                "lon": day_data.geometry.centroid.x.mean()},
                               opacity=0.5)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

plot_interactive_map(2023, 11, 9)

In [ ]:
import plotly.express as px
import geopandas as gpd
import pandas as pd

file_path = '/content/drive/Shareddrives/dm_final/dm_final/data/test_gridcentroid_test_07x07_weekday.csv.xls'
data = pd.read_csv(file_path)

# Load the data from both CSV files
grid_mapping_path = '/content/drive/Shareddrives/dm_final/dm_final/data/grid_mapping_7x7.csv'
gridcentroid_path = '/content/drive/Shareddrives/dm_final/dm_final/data/test_gridcentroid_test_07x07_weekday.csv.xls'

grid_mapping = pd.read_csv(grid_mapping_path)
gridcentroid = pd.read_csv(gridcentroid_path)

# Understanding structure
grid_mapping_head = grid_mapping.head()
gridcentroid_head = gridcentroid.head()

grid_mapping_head, gridcentroid_head

# Merging the two DataFrames on the 'GridID' column
data = pd.merge(gridcentroid, grid_mapping, on='GridID', how='left')

data_head = data.head()
print(data_head)

# # Converting the 'geometry' column to a GeoDataFrame
data['geometry'] = gpd.GeoSeries.from_wkt(data['geometry'])
geo_data = gpd.GeoDataFrame(data, geometry='geometry')

# Function to create an interactive map for a selected date
def plot_interactive_map(selected_year, selected_month, selected_day):
    day_data = geo_data[(geo_data['OccurrenceYear'] == selected_year) &
                        (geo_data['OccurrenceMonth'] == selected_month) &
                        (geo_data['OccurrenceDatePart'] == selected_day)]

    fig = px.choropleth_mapbox(day_data, geojson=day_data.geometry.__geo_interface__,
                               locations=day_data.index, color='NumberOfIncidents',
                               color_continuous_scale="OrRd",
                               mapbox_style="carto-positron",
                               zoom=10, center={"lat": day_data.geometry.centroid.y.mean(),
                                                "lon": day_data.geometry.centroid.x.mean()},
                               opacity=0.5)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

plot_interactive_map(2023, 11, 9)

   GridNumber  OccurrenceYear  OccurrenceMonth  OccurrenceDatePart  IsWeekend  \
0         1.0            2023               10                  23          0   
1         1.0            2023               10                  24          0   
2         1.0            2023               10                  25          0   
3         1.0            2023               10                  27          1   
4         1.0            2023               10                  28          1   

   NumberOfIncidents  GridID  X_centroid  Y_centroid  \
0                  2       1 -111.971255   33.330417   
1                  1       1 -111.971255   33.330417   
2                  3       1 -111.971255   33.330417   
3                  1       1 -111.971255   33.330417   
4                  1       1 -111.971255   33.330417   

                                            geometry  
0  POLYGON ((-111.96403442971423 33.3200400060000...  
1  POLYGON ((-111.96403442971423 33.3200400060000...  
2  POLYGON 

In [ ]:
import plotly.express as px
import geopandas as gpd
import pandas as pd

file_path = '/content/drive/Shareddrives/dm_final/dm_final/data/final_predicted_grid.csv'
data = pd.read_csv(file_path)

# Converting the 'geometry' column to a GeoDataFrame
data['geometry'] = gpd.GeoSeries.from_wkt(data['geometry'])
geo_data = gpd.GeoDataFrame(data, geometry='geometry')

# Function to create an interactive map for a selected date
def plot_interactive_map(selected_year, selected_month, selected_day):
    day_data = geo_data[(geo_data['OccurrenceYear'] == selected_year) &
                        (geo_data['OccurrenceMonth'] == selected_month) &
                        (geo_data['OccurrenceDatePart'] == selected_day)]

    # Custom color scale: Green to Red
    # custom_color_scale = [("green", 0), ("yellow", 0.5), ("red", 1)]

    fig = px.choropleth_mapbox(day_data, geojson=day_data.geometry.__geo_interface__,
                               locations=day_data.index, color='PredictedNumberOfIncidents',
                               color_continuous_scale="icefire",
                               mapbox_style="carto-positron",
                               zoom=10, center={"lat": day_data.geometry.centroid.y.mean(),
                                                "lon": day_data.geometry.centroid.x.mean()},
                               opacity=0.5)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

plot_interactive_map(2023, 11, 11)